In [ ]:
##
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field

from unskript.connectors.aws import aws_get_paginator


from beartype import beartype
@beartype
def aws_filter_ec2_by_tags(handle, tag_key: str, tag_value: str, region: str):
    """aws_filter_ec2_by_tags Returns an array of instances matching tags.

        :type nbParamsObj: object
        :param nbParamsObj: Object containing global params for the notebook.

        :type credentialsDict: dict
        :param credentialsDict: Dictionary of credentials info.

        :type inputParamsJson: string
        :param inputParamsJson: Json string of the input params.

        :rtype: Array of instances matching tags.
    """
    # Input param validation.

    ec2Client = handle.client('ec2', region_name=region)
    res = aws_get_paginator(ec2Client, "describe_instances", "Reservations",
                            Filters=[{'Name': 'tag:' + tag_key, 'Values': [tag_value]}])

    result = []
    for reservation in res:
        for instance in reservation['Instances']:
            result.append(instance['InstanceId'])
    return result


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "region": "region",
    "tag_key": "tag_name",
    "tag_value": "tag_value"
    }''')

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.output = task.execute(aws_filter_ec2_by_tags, hdl=hdl, args=args)
    if task.output_name != None:
        globals().update({task.output_name: task.output[0]})

if hasattr(task, 'output'):
    if isinstance(task.output, (list, tuple)):
        for item in task.output:
            print(f'item: {item}')
    elif isinstance(task.output, dict):
        for item in task.output.items():
            print(f'item: {item}')
    else:
        print(f'Output for {task.name}')
        print(task.output)
    w.tasks[task.name]= task.output

In [ ]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##

import enum
from pydantic import BaseModel, Field
from typing import Optional, List
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from unskript.legos.aws.aws_get_handle import Session


class StatisticsType(str, enum.Enum):
    SAMPLE_COUNT = "SampleCount"
    AVERAGE = "Average"
    SUM = "Sum"
    MINIMUM = "Minimum"
    MAXIMUM = "Maximum"


from beartype import beartype
@beartype
def aws_get_cloudwatch_statistics(
    hdl: Session,
    name_space: str,
    metric_name: str,
    dimensions: List[dict],
    timeSince: int,
    statistics: str,
    region: str,
    period: int = 60,
):
    """aws_get_cloud_statistics shows ploted AWS cloudwatch statistics.
    for a given instance ID. This routine assume instance_id
    being present in the inputParmsJson.

    :type name_space: string
    :param name_space: he namespace of the metric, with or without spaces. For eg: AWS/SQS, AWS/ECS

    :type metric_name: string
    :param metric_name: The name of the metric, with or without spaces.

    :type dimensions: List[dict]
    :param dimensions: A dimension is a name/value pair that is part of the identity of a metric.

    :type period: int
    :param period: The granularity, in seconds, of the returned data points.

    :type timeSince: int
    :param timeSince: Starting from now, window (in seconds) for which you want to get the datapoints for.

    :type statistics: string
    :param statistics: Cloudwatch metric statistics. Possible values: SampleCount, Average, Sum, Minimum, Maximum.

    :type region: string
    :param region: AWS Region of the cloudwatch.

    :rtype: Shows ploted statistics.
    """

    cloudwatchClient = hdl.client("cloudwatch", region_name=region)
    # Gets metric statistics.
    res = cloudwatchClient.get_metric_statistics(
        Namespace=name_space,
        MetricName=metric_name,
        Dimensions=dimensions,
        Period=period,
        StartTime=datetime.utcnow() - timedelta(seconds=timeSince),
        EndTime=datetime.utcnow(),
        Statistics=[statistics],
    )

    data = {}
    for datapoints in res["Datapoints"]:
        data[datapoints["Timestamp"]] = datapoints[statistics]

    # Sorts data.
    data_keys = data.keys()
    times_stamps = list(data_keys)
    times_stamps.sort()
    sorted_values = []
    for value in times_stamps:
        sorted_values.append(data[value])

    # Puts datapoints into the plot.
    plt.plot_date(times_stamps, sorted_values, "-o")

    return


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "dimensions": "iter_item",
    "metric_name": "\\"CPUUtilization\\"",
    "name_space": "\\"AWS/EC2\\"",
    "period": "60",
    "region": "region",
    "statistics": "\\"Average\\"",
    "timeSince": "36000"
    }''')

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.output = task.execute(aws_get_cloudwatch_statistics, hdl=hdl, args=args)
    if task.output_name != None:
        globals().update({task.output_name: task.output[0]})

if hasattr(task, 'output'):
    if isinstance(task.output, (list, tuple)):
        for item in task.output:
            print(f'item: {item}')
    elif isinstance(task.output, dict):
        for item in task.output.items():
            print(f'item: {item}')
    else:
        print(f'Output for {task.name}')
        print(task.output)
    w.tasks[task.name]= task.output

In [ ]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from typing import List
from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def aws_restart_ec2_instances(handle, instance_ids: List, region: str):
    """aws_restart_instances Restarts instances.

        :type nbParamsObj: object
        :param nbParamsObj: Object containing global params for the notebook.

        :type credentialsDict: dict
        :param credentialsDict: Dictionary of credentials info.

        :type inputParamsJson: string
        :param inputParamsJson: Json string of the input params.

        :rtype: Dict with the stopped instances state info.
    """

    ec2Client = handle.client('ec2', region_name=region)
    res = ec2Client.reboot_instances(InstanceIds=instance_ids)

    return res


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "instance_ids": "iter_item",
    "region": "region"
    }''')

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.output = task.execute(aws_restart_ec2_instances, hdl=hdl, args=args)
    if task.output_name != None:
        globals().update({task.output_name: task.output[0]})

if hasattr(task, 'output'):
    if isinstance(task.output, (list, tuple)):
        for item in task.output:
            print(f'item: {item}')
    elif isinstance(task.output, dict):
        for item in task.output.items():
            print(f'item: {item}')
    else:
        print(f'Output for {task.name}')
        print(task.output)
    w.tasks[task.name]= task.output

In [ ]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##

import pprint

from pydantic import BaseModel, Field
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError

pp = pprint.PrettyPrinter(indent=2)


from beartype import beartype
def legoPrinter(func):
    def Printer(*args, **kwargs):
        output = func(*args, **kwargs)
        if output:
            channel = kwargs["channel"]
            pp.pprint(print(f"Message sent to Slack channel {channel}"))
        return output
    return Printer


@legoPrinter
@beartype
def slack_post_message(
        handle: WebClient,
        channel: str,
        message: str) -> bool:

    try:
        response = handle.chat_postMessage(
            channel=channel,
            text=message)
        return True
    except SlackApiError as e:
        print("\n\n")
        pp.pprint(
            f"Failed sending message to slack channel {channel}, Error: {e.response['error']}")
        return False
    except Exception as e:
        print("\n\n")
        pp.pprint(
            f"Failed sending message to slack channel {channel}, Error: {e.__str__()}")
        return False


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "channel": "channel",
    "message": "f\\"Restarted instances {inst_list}\\""
    }''')

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.output = task.execute(slack_post_message, hdl=hdl, args=args)
    if task.output_name != None:
        globals().update({task.output_name: task.output[0]})